# 4.2 데이터 전처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

In [ ]:
# 추가 라이브러리 임포트
import seaborn as sns

# 데이터 읽어 들이기
df_titanic = sns.load_dataset("titanic")

# 우리말 필드명 정의
columns_t = [ '생존', '선실등급', '성별', '연령', '형제_및_배우자_수',
              '부모_및_자녀_수', '요금', '승선항_코드', '선실등급명',
              '남녀어린이_구분', '성인_남성_여부', '데크번호', '승선항', '생존_여부', '독신_여부']
df_titanic.columns = columns_t

#### 데이터 확인

In [ ]:
display(df_titanic.head())

In [ ]:
print(df_titanic.shape)

### 4.2.1 불필요한 필드 삭제하기

In [ ]:
# 중복되는 필드를 제거한다

# "선실등급명" ("선실등급"과 중복)
df1 = df_titanic.drop('선실등급명', axis=1)

# "승선항" ("승선항 코드"와 중복)
df2 = df1.drop('승선항', axis=1)

# "생존 여부" ("생존"과 중복)
df3 = df2.drop('생존_여부', axis=1)

# 결과 확인
display(df3.head())

### 4.2.2 누락 값 처리하기

In [ ]:
# 누락 값 확인하기
display(df3.isnull().sum())

In [ ]:
display(df3['데크번호'].value_counts())

#### 누락 값 처리 방법

**승선항 코드**: 누락 값이 2건(적음)
-> 행 전체를 삭제 

**연령**: 숫자 데이터이며 누락 값이 177건(꽤 많음)
-> 필드 값의 평균값으로 누락 값을 대체

**데크번호**: 코드값 필드이며 누락 값이 688건(아주 많음)
-> 누락 값을 의미하는 더미 코드를 부여함

In [ ]:
# 승선항_코드: 누락 값이 2건(적음)
# -> 행 전체를 삭제

# dropna 함수를 사용
df4 = df3.dropna(subset = ['승선항_코드'])

# 연령: 숫자 데이터이며 누락 값이 177건(꽤 많음)
# -> 필드 값의 평균값으로 누락 값을 대체

# 평균값 계산
age_average = df4['연령'].mean()

# fillna 함수를 사용
df5 = df4.fillna({'연령': age_average})

# 데크번호: 코드값 필드이며 누락 값이 688건(아주 많음)
# -> 누락 값을 의미하는 더미 코드를 부여함

# replace 함수를 사용 (더미코드는 N)
#df6 = df5.replace({'데크번호': {np.nan: 'N'}})
df5['데크번호'] = df5['데크번호'].astype(np.str)
df6 = df5.fillna({'데크번호' : 'N'})

In [ ]:
# 결과 확인
display(df6.isnull().sum())

display(df6.head())

### 4.2.3 이진 레이블값 필드를 숫자 값 필드로 만들기

**성별**: male / female  
**성인_남성_여부**: True / False  
**독신_여부**: True / False  
    
위 필드의 값을 각각 1 / 0으로 변환

#### '성별' 필드를 숫자 값 필드로 변환

In [ ]:
display(df6['성별'].value_counts())

In [ ]:
# 딕셔너리 mf_map을 정의
mf_map = {'male': 1, 'female': 0}

# map 함수를 사용해 레이블을 수치로 변환
df7 = df6.copy()
df7['성별'] = df7['성별'].map(mf_map)

# 결과 확인
display(df7.head())

#### '성인_남성_여부', '독신_여부' 필드를 숫자 값 필드로 변환

In [ ]:
display(df7['성인_남성_여부'].value_counts())

In [ ]:
# 딕셔너리 tf_map을 정의
tf_map = {True: 1, False: 0}

# map 함수를 사용해 레이블을 수치로 변환
df8 = df7.copy()
df8['성인_남성_여부'] = df8['성인_남성_여부'].map(tf_map)

# map 함수를 사용해 레이블을 수치로 변환
df9 = df8.copy()
df9['독신_여부'] = df8['독신_여부'].map(tf_map)

# 결과 확인
display(df9.head())

### 4.2.4 다중 레이블값 필드를 숫자 값 필드로 만들기
원-핫 인코딩

In [ ]:
# 변환 전
display(df9[['남녀어린이_구분']].head(10))

In [ ]:
# get_dummies 함수의 사용 예제

w = pd.get_dummies(df9['남녀어린이_구분'], prefix='남녀어린이_구분')
display(w.head(10))

In [ ]:
# get_dummies 함수를 사용해 정의한
# 범주값을 원-핫 벡터로 변환하는 함수
# df 대상 데이터프레임
# column 대상 필드

def enc(df, column):
    # 원-핫 벡터 생성
    df_dummy = pd.get_dummies(df[column], prefix=column)
    # 기존 필드 삭제
    df_drop = df.drop([column], axis=1)
    # 데이터프레임과 원-핫 인코딩 필드를 결합
    df1 = pd.concat([df_drop,df_dummy],axis=1)
    return df1

#### 남녀어린이_구분
man / woman / child

In [ ]:
# 레이블값 확인하기
display(df9['남녀어린이_구분'].value_counts())

In [ ]:
# 원-핫 인코딩 적용

# 남녀어린이_구분 필드
df10 = enc(df9, '남녀어린이_구분')

# 결과 확인하기
display(df10.head())

#### 승선항 코드 및 데크번호

In [ ]:
# 원-핫 인코딩 적용

# 승선항_코드 필드
df11 = enc(df10, '승선항_코드')

# 데크번호 필드
df12 = enc(df11, '데크번호')

# 결과 확인하기
display(df12.head())

### 4.2.5 데이터 정규화

In [ ]:
# 정규화

df13 = df12.copy()
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
df13[['연령', '요금']] = stdsc.fit_transform(df13[['연령', '요금']])

# 결과 확인하기
display(df13.head())